# LLM with Web Search and Crawl

Code to crawl the top n pages of a Google search result and serve them to LLM in order to utilize rich context.



In [3]:
import re
import requests
import sys
import os
from openai import AzureOpenAI
import tiktoken
from dotenv import load_dotenv
load_dotenv(override=True) 

client = AzureOpenAI(
  azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"), 
  api_key=os.getenv("AZURE_OPENAI_KEY"),  
  api_version="2024-08-01-preview"
)

CHAT_COMPLETIONS_MODEL = os.getenv('AZURE_OPENAI_DEPLOYMENT_NAME')

bs4 or scrapy?

In [5]:
import requests
import json
import scrapy
from bs4 import BeautifulSoup
import httpx
import asyncio
from urllib.parse import urljoin

GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
GOOGLE_CSE_ID = os.getenv("GOOGLE_CSE_ID")

def extract_text_and_tables_by_bs4(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")
    # Extract main text
    paragraphs = [p.get_text().strip() for p in soup.find_all("p") if p.get_text().strip()]
    text = "\n".join(paragraphs)
    return text


async def extract_text_and_tables_async(url):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"
    }
    async with httpx.AsyncClient(timeout=3, follow_redirects=True) as client:
        try:
            response = await client.get(url, headers=headers)
            response.raise_for_status()
        except httpx.HTTPStatusError as e:
            # Handle 302 redirect manually if follow_redirects fails
            if e.response.status_code == 302 and "location" in e.response.headers:
                redirect_url = e.response.headers["location"]
                if not redirect_url.startswith("http"):
                    # handle relative redirects
                    redirect_url = urljoin(url, redirect_url)
                try:
                    response = await client.get(redirect_url, headers=headers)
                    response.raise_for_status()
                except Exception as e2:
                    print(f"Redirect request failed: {e2}")
                    return ""
            else:
                print(f"Request failed: {e}")
                return ""
        except httpx.HTTPError as e:
            print(f"Request failed: {e}")
            return ""

        selector = scrapy.Selector(text=response.text)
        paragraphs = [p.strip() for p in selector.css('p::text').getall() if p.strip()]
        text = "\n".join(paragraphs)
        return text

async def add_context_async(top_urls = []):
    async def gather_contexts():
        tasks = [extract_text_and_tables_async(url) for url in top_urls]
        results = await asyncio.gather(*tasks)
        return results
    return await gather_contexts()

def google_search(query, num=5):
    url = "https://www.googleapis.com/customsearch/v1"
    params = {
        "q": query,
        "key": GOOGLE_API_KEY,
        "cx": GOOGLE_CSE_ID,
        "num": num, 
        "locale": "ko",  # 한국어로 검색
        "siteSearch": "samsung.com",
        "siteSearchFilter": "e",
    }
    response = requests.get(url, params=params)
    results = response.json()
    return results.get("items", [])

       
QUERY_REWRITE_PROMPT = """
            <<지시문>>
            너는 구글 검색과 LLM 질의 최적화 전문가야. 사용자가 입력한 질문을 두 가지 목적에 맞게 재작성해.

            1. Google Search용 Query Rewrite:
            - 사용자의 질문을 실제 구글 검색창에 입력할 수 있도록, 명확하고 간결한 핵심 키워드 중심의 검색어로 재작성해.
            - 불필요한 문장, 맥락 설명은 빼고, 검색에 최적화된 형태로 만들어.
            - 핵심 키워드를 반복적으로 사용해 검색의 정확도를 높여.

            2. LLM Query용 Rewrite:
            - 사용자의 질문을 LLM이 더 잘 이해하고 답변할 수 있도록, 맥락과 의도를 명확히 드러내는 자연스러운 문장으로 재작성해.
            - 필요한 경우 추가 설명이나 세부 조건을 포함해서 질문의 목적이 분명히 드러나도록 만들어.
            - LLM이 답변에 집중할 수 있도록 핵심 단어를 반복 사용해.

            <<예시>>
            * 질문: 삼성전자 제품 중 2구 말고 다른 인덕션 추천해줘
            * 구글 검색용 재작성: 삼성전자 3구 이상 인덕션 추천
            * LLM 답변용 재작성: 삼성전자 인덕션 중 2구 모델이 아닌, 3구 이상 또는 다양한 화구 수를 가진 다른 인덕션 제품을 추천해 주세요. 각 모델의 주요 기능과 장점도 함께 알려주세요.

            <<질문>>
            {user_query}

            <<출력포맷>>
            반드시 아래와 같이 json 형식으로 출력해.
            {"google_search": "구글 검색용 재작성", "llm_query": "LLM 답변용 재작성"}
        """     
  
def rewrite_query_for_search_and_llm(query, client: AzureOpenAI):
        response = client.chat.completions.create(
            model=CHAT_COMPLETIONS_MODEL,
            messages=[
                {"role": "system", "content": QUERY_REWRITE_PROMPT},
                {"role": "user", "content": query}
            ],
            temperature=0.8,
            max_tokens=300,
            response_format= {"type": "json_object"},
        )
        
        return json.loads(response.choices[0].message.content.strip())







In [6]:
from IPython.display import Markdown, display
from datetime import datetime

RESULTS_COUNT = 5

inputs = [
    "삼성전자 제품 중 2구 말고 다른 인덕션 추천해줘",
    "부모님에게 선물하고 싶은데 삼성전자 TV 추천해줘",
    "삼성전자 25년 제품이 작년 대비 좋아진것은",
    "삼성전자 JBL과 하만카돈 차이점이 뭐야",
    "갤럭시 버즈 이어버드 한쪽을 새로 구매했는데 페어링 어떻게 하나요",
    "삼성전자 S25 무게가 S24와 비교 했을때 얼마나 차이나"
]

#TODO 날씨나 뉴스, 기타 다른 특정정보는 Function Call
# inputs = ["날씨, 뉴스"] ##

for input in inputs:
    print(f"Original Input: {input}")
    
    query_rewrite = rewrite_query_for_search_and_llm(input, client)
    print(f"Google Search Query: {query_rewrite['google_search']}")
    print(f"LLM Query: {query_rewrite['llm_query']}")

    results = google_search(query_rewrite['google_search'], RESULTS_COUNT)
    print(f"Google Search Results: {len(results)}")
    top_urls = [results[i]["link"] for i in range(len(results))]
    contexts = await add_context_async(top_urls)

    # for i, context in enumerate(contexts):
    #     print(f"Context {i+1}: {context}...")  # Print first 1000 chars of each context
    #     print("\n--- End of Context ---\n")

    now = datetime.now()
    year = now.year
    month = now.month
    day = now.day

    system_prompt = "너는 삼성전자 제품 관련 정보를 제공하는 챗봇이야. 답변은 마크다운으로 작성해줘."
    user_prompt = f"""
        너는 아래 제공하는 구글에서 검색한 컨텍스트를 바탕으로 질문에 대한 답변을 제공해야 해. 컨텍스트를 최대한 활용하여 풍부하게 답변을 해야해. 
        현재는 {year}년 {month}월 {day}일이므로 최신의 데이터를 기반으로 답변을 해줘.

        구글에서 제공한 컨텍스트: {contexts}
        질문: {query_rewrite['llm_query']}
        """

    response = client.chat.completions.create(
        model=CHAT_COMPLETIONS_MODEL,
        messages=[{"role": "system", "content": system_prompt},
                 {"role": "user", "content": user_prompt}],
        top_p=0.9,
        max_tokens=1500
    )

    display(Markdown(response.choices[0].message.content))

Original Input: 삼성전자 제품 중 2구 말고 다른 인덕션 추천해줘
Google Search Query: 삼성전자 3구 인덕션 추천
LLM Query: 삼성전자 인덕션 제품 중 2구 모델이 아닌 3구 이상 또는 다양한 화구 수를 가진 인덕션 제품을 추천해 주세요. 각 모델의 주요 기능과 장점도 알려주세요.
Google Search Results: 5


안녕하세요! 2025년 5월 기준으로 삼성전자 인덕션 3구 이상 모델 중 인기 있고 추천할 만한 제품을 주요 기능과 장점과 함께 정리해 드리겠습니다.

---

## 1. 삼성전자 비스포크 AI 인덕션 3구 모델 - NZ63DB607CF (2024년형)

### 주요 기능 및 특징
- **최고 출력**: 대화구 3,400W / 중화구 2,600W / 소화구 1,200W의 강력한 화력 (상위 10% 이내)
- **AI 넘침 방지 모드** 탑재로 요리 중 넘침 걱정 최소화
- **저소음 DCN 기능**으로 조용한 요리 환경 제공
- **대화형 알림창 및 심플한 1개 조작부**로 사용 편의성 극대화
- **프레임리스 디자인 & 얇은 두께**로 설치 시 공간 효율 우수
- 가격이 100만 원 미만으로 가성비 개선

### 장점
- 강력한 화력으로 빠른 조리 가능
- 편의 기능 대폭 강화(알림창, AI 넘침방지, 저소음)
- 심플하고 세련된 디자인
- 가격 대비 성능과 기능 우수

---

## 2. 삼성전자 비스포크 3구 인덕션 - NZ63D650BXE (하위 모델)

### 주요 기능 및 특징
- 대화구 3,300W / 중화구 2,600W / 소화구 1,200W 출력 동일
- AI 끓음 감지 기능 미탑재, 조작부는 각 화구별 3개 탑재
- 두께가 조금 더 두껍지만 가격이 66~79만원대로 합리적

### 장점
- 기본 화력과 성능 충실
- 가성비 좋은 3구 인덕션
- 예산 절감형 모델로 추천

---

## 3. 삼성전자 인덕션 3구 가성비 모델 - NZ63B4026AK

### 주요 기능 및 특징
- 대화구 3,300W / 중화구 2,600W / 소화구 1,200W 출력 제공
- 블랙 상판, 대화형 알림창 없음
- 가격 55~65만원으로 삼성 3구 인덕션 중 최고 가성비

### 장점
- 뛰어난 출력 대비 매우 합리적인 가격
- 필수 기능 충실, 간단한 디자인 선호 시 적합

---

## 4. 삼성전자 비스포크 AI 인덕션 플렉스존 3구 - NZ63DB657CA*S (프리미엄 모델)

### 주요 기능 및 특징
- 4분할 플렉스존으로 다양한 크기 용기 사용 가능
- 각 화구 일반 출력 2,000W 이상으로 최상급 화력
- LCD 대화형 알림창, AI 끓음 감지, 저소음 DCN 기능 탑재
- 앱 연동 원격 제어 가능
- 얇아진 두께와 세련된 테두리 마감

### 장점
- 프리미엄급 화력과 다양한 편의 기능 제공
- 사용자 맞춤형 조리 공간 제공하는 플렉스존
- 합리적인 프리미엄 가격대

---

## 5. 삼성전자 더 플레이트 인덕션 2구 모델 (참고용)

- 2구 제품이지만 디자인과 출력이 뛰어나고 4.6cm의 얇은 두께
- 3,300W 출력으로 빠른 조리 가능
- 20만원대 고급형으로 좁은 공간에 적합

---

# 요약 및 추천

| 모델명                  | 화구 수 | 최고 출력 (대/중/소) | 주요 기능                           | 가격대        | 추천 대상                          |
|----------------------|-------|-------------------|----------------------------------|------------|-------------------------------|
| NZ63DB607CF (비스포크 AI) | 3구    | 3,400W / 2,600W / 1,200W | AI 넘침 방지, 저소음, 알림창, 프레임리스 | 100만원 미만 | 고성능+편의기능 중시 소비자           |
| NZ63D650BXE (비스포크)   | 3구    | 3,300W / 2,600W / 1,200W | 기본 화력, 개별 조작부           | 66~79만원    | 예산 절약형, 기본 기능 선호           |
| NZ63B4026AK             | 3구    | 3,300W / 2,600W / 1,200W | 기본 충실, 가성비 최고          | 55~65만원    | 가성비 중시, 심플한 기능 사용자         |
| NZ63DB657CA*S (프리미엄) | 3구    | 2,000W 이상 모든 화구     | 4분할 플렉스존, AI 끓음 감지, 원격제어 | 100만원대 초반 | 프리미엄 성능+스마트 기능 원하는 소비자  |

---

필요한 용도, 예산, 편의 기능에 따라 위 모델 중 선택하시면 좋겠습니다. 삼성전자 비스포크 라인업은 디자인과 편의 기능, 화력에서 모두 우수해 인기 많으니 참고하세요!

더 궁금한 점 있으면 언제든 문의해 주세요! 😊

---

### 참고
- 모든 정보는 2025년 5월 최신 기준입니다.
- 설치 및 추가 부대비용은 별도입니다.  
- 제품 구매 시 공식 삼성전자 판매처 및 신뢰할 수 있는 판매처 이용 권장드립니다.

Original Input: 부모님에게 선물하고 싶은데 삼성전자 TV 추천해줘
Google Search Query: 삼성전자 TV 추천 부모님 선물
LLM Query: 부모님께 드릴 선물로 적합한 삼성전자 TV 모델을 추천해 주세요. 사용 환경과 주요 기능을 고려한 삼성전자 TV 추천을 부탁드립니다.
Google Search Results: 5
Request failed: 


안녕하세요! 2025년 5월 기준으로 부모님께 드릴 삼성전자 TV를 추천드리겠습니다.

---

## 추천 모델 및 이유

### 1. 삼성 QLED 70~75인치 라인업 (예: QD70, QD67, QD68 등)
- **사이즈**: 70~75인치로, 거거익선 원칙에 부합하며 큰 화면으로 자연 다큐나 여행기 감상에 적합합니다.
- **화질**: QLED 패널을 사용해 LED보다 우수한 색 재현력과 밝기를 제공합니다.
- **사용 편의성**: 삼성 스마트 TV의 직관적 UI와 국내 대기업의 안정적인 A/S 지원.
- **가격대**: 130만원 내외(할인 및 프로모션 시 더 저렴할 수 있음).

### 2. 삼성 그랑데 드럼세탁기, 코드제로 청소기 등과 함께 가전 라인업 연계
- 부모님 댁 가전 선물에 이미 삼성전자 제품 경험이 있으시다면 TV도 같은 브랜드를 선호하실 가능성이 높습니다.

---

## 사용 환경 및 고려사항

- **화질 중시**: 자연 다큐, 여행기 영상 감상에 최적화된 QLED TV가 적합합니다.
- **브랜드 신뢰성**: 부모님 세대는 브랜드 신뢰도가 매우 중요하므로 삼성전자 제품 추천드립니다.
- **A/S 편리성**: 삼성전자 코리아 서비스망이 전국적으로 잘 구축되어 있어, 고장 시 신속한 지원이 가능합니다.
- **예산**: 약 130만원 범위 내에서 최적의 모델을 선택하되, 할인 기간을 노리면 더 좋은 조건으로 구매 가능.

---

## 구매 팁

- **할인 시즌 활용**: 연말, 설 명절, 블랙프라이데이 등 대규모 할인 기간에 구매하면 좋은 조건을 얻을 수 있습니다.
- **쿠팡, 네이버 쇼핑 등 온라인 몰 자주 확인**: 가격 변동 및 핫딜 정보를 수시로 모니터링하세요.
- **모델별 실제 사용 후기 참고**: QLED 70인치 이상 모델 중 최근 출시 제품 위주로 후기 확인을 권장드립니다.

---

## 요약

| 항목            | 추천 내용                           |
|-----------------|-----------------------------------|
| 모델명          | 삼성 QLED 70~75인치 (QD70, QD67, QD68 등) |
| 주요 특징       | 우수한 화질(QLED), 큰 화면, 스마트 TV, 국내 A/S 우수 |
| 예산            | 약 130만원 내외                   |
| 구매 시기 팁    | 할인 시즌(연말, 명절 등) 활용     |
| 브랜드 신뢰도   | 부모님 세대에 적합, 삼성전자 강력 추천 |

---

필요하시면 구체적인 최신 모델명과 상세 스펙, 구매 링크도 안내해 드릴 수 있으니 편하게 문의 주세요!  
부모님께서 만족하실 수 있는 좋은 선물 되시길 바랍니다. 😊

Original Input: 삼성전자 25년 제품이 작년 대비 좋아진것은
Google Search Query: 삼성전자 2025년 제품 작년 대비 개선점
LLM Query: 삼성전자 2025년 제품이 작년 제품과 비교했을 때 어떤 점에서 좋아졌는지 주요 개선점과 기능 향상을 상세히 설명해 주세요.
Google Search Results: 5


...

삼성전자에서 판매하는 **JBL**과 **하만카돈(Harman Kardon)** 스피커는 모두 하만(Harman)이 소유한 브랜드로, 각기 다른 특성과 타깃층을 가지고 있습니다. 두 브랜드의 음질, 디자인, 가격, 기술적 특징을 비교해 드리겠습니다.

---

## 1. 음질 비교

- **JBL**
  - 대체로 강력한 베이스와 다이나믹한 사운드를 특징으로 합니다.
  - 라이브 음악이나 파티용으로 적합한 강한 저음과 명확한 중고음이 특징입니다.
  - 실내외 어디서든 활기차고 에너지 넘치는 사운드를 원하는 사용자에게 인기가 많습니다.

- **하만카돈**
  - 좀 더 정교하고 세련된 사운드를 제공합니다.
  - 고음역대의 디테일과 중음의 풍부함에 집중하여 음악의 깊이와 공간감을 강조합니다.
  - 클래식, 재즈, 보컬 음악 감상에 적합한 부드럽고 균형 잡힌 음질이 특징입니다.

---

## 2. 디자인

- **JBL**
  - 스포티하고 대담한 디자인이 많으며, 강렬한 컬러와 내구성이 좋은 소재 사용이 두드러집니다.
  - 휴대성을 고려한 콤팩트한 크기와 방수 기능(특히 플립, 펄스 시리즈)도 많아 야외 활동에 적합합니다.

- **하만카돈**
  - 고급스러운 미니멀리즘 디자인이 특징으로, 금속 재질과 부드러운 곡선 형태를 많이 사용합니다.
  - 인테리어 소품처럼 집안 어디에 두어도 세련된 분위기를 연출할 수 있는 스타일을 지향합니다.

---

## 3. 가격대

- **JBL**
  - 보통 중저가부터 중고가까지 폭넓은 가격대의 제품이 많습니다.
  - 합리적인 가격에 뛰어난 성능과 휴대성을 원하는 소비자에게 적합합니다.

- **하만카돈**
  - 중고가에서 고가 제품 중심으로 프리미엄 가격대를 형성합니다.
  - 고품질 사운드와 세련된 디자인에 투자할 의향이 있는 사용자에게 어울립니다.

---

## 4. 기술적 특징

- **JBL**
  - 블루투스 연결이 강력하며, 일부 제품은 IPX7 방수 등급을 지원해 야외 사용에 용이합니다.
  - JBL Connect+ 기능으로 여러 대의 스피커를 무선으로 연결 가능.
  - 배터리 사용시간이 길고, 빠른 충전 기능을 탑재한 제품도 많습니다.

- **하만카돈**
  - 고해상도 오디오 지원, DSP(Digital Signal Processing) 기술로 정교한 사운드 튜닝.
  - 일부 모델에는 멀티룸 오디오 기능과 스마트 스피커 기능(음성 인식 등)이 탑재되어 있습니다.
  - 고급 재료와 설계로 왜곡을 최소화하여 클리어한 음질을 구현합니다.

---

## 요약

| 구분       | JBL                             | 하만카돈                         |
|------------|--------------------------------|---------------------------------|
| 음질       | 강한 베이스, 다이나믹한 사운드   | 정교하고 세련된, 균형 잡힌 음질   |
| 디자인     | 스포티하고 대담한 스타일          | 고급스럽고 미니멀한 디자인        |
| 가격대     | 중저가~중고가                   | 중고가~고가                      |
| 기술 특징  | 방수, 긴 배터리, JBL Connect+  | 고해상도 오디오, DSP, 멀티룸 지원 |

---

### 결론
- **활기차고 강한 저음, 휴대성과 내구성을 중시한다면 JBL 스피커를 추천합니다.**
- **고급스러운 음질과 디자인, 집에서 깊이 있는 음악 감상을 원한다면 하만카돈 스피커가 더 적합합니다.**

필요한 용도와 예산에 맞춰 선택하시면 좋겠습니다! 삼성전자 매장이나 공식 온라인몰에서 최신 모델과 가격 정보를 확인하실 수 있습니다.

Original Input: 갤럭시 버즈 이어버드 한쪽을 새로 구매했는데 페어링 어떻게 하나요
Google Search Query: 갤럭시 버즈 이어버드 한쪽만 구매 페어링 방법
LLM Query: 갤럭시 버즈 이어버드 한쪽만 새로 구매했을 때, 새 이어버드를 기존 기기와 어떻게 페어링하는지 단계별 방법을 알려주세요. 한쪽 이어버드 페어링 과정과 주의사항도 포함해 주세요.
Google Search Results: 5


안녕하세요! 갤럭시 버즈 이어버드 한쪽만 새로 구매하여 기존 기기와 페어링하는 방법을 단계별로 안내해드리겠습니다. 아래 내용을 참고해 주세요.

---

## 갤럭시 버즈 이어버드 한쪽만 새로 구매 후 기존 기기와 페어링하는 방법

### 1. 준비사항
- 새로 구매한 이어버드가 기존 이어버드와 같은 **배터리 타입(종류)**인지 반드시 확인하세요. (배터리 타입이 다르면 커플링이 불가합니다.)
- 이어버드와 충전 케이스가 정상 작동하는지 확인하세요.
- 갤럭시 웨어러블(Galaxy Wearable) 앱이 설치된 스마트폰 준비

---

### 2. 이어버드 커플링 기본 절차

#### ① 양쪽 이어버드 충전 및 케이스에 넣기
- 이어버드 한 쪽씩 충전 케이스의 좌우 슬롯 홈에 맞게 넣으세요.
- 이어버드가 충전 단자에 정상적으로 안착되어 충전 케이스 LED가 녹색 또는 빨간색으로 깜빡여야 합니다.

#### ② 이어버드 동시에 터치로 커플링 시도 (셀프 커플링)
- 양쪽 이어버드를 동시에 7초 이상 길게 터치하세요.
- 이어버드 배터리 상태 표시등이 녹색으로 깜빡인 후 꺼지면 커플링 완료입니다.
- 만약 배터리 표시등이 빨간색으로 계속 깜빡이면, 이어버드를 충전 케이스에서 뺐다가 다시 넣고 7초 이상 터치 과정을 반복하세요.

---

### 3. 충전 케이스 버튼 이용 커플링

- 충전 케이스를 열어둔 상태에서 케이스 아래 충전 단자 옆에 있는 **연결 버튼**을 7초 이상 누르세요.
- 충전 케이스 LED가 파란색으로 깜빡인 후 꺼지면 커플링이 완료됩니다.
- LED가 빨간색으로 계속 깜빡이면 단자 통신 오류일 수 있으니 1단계부터 다시 시도하거나 단자 접점에 이물질, 부식, 침수 여부를 점검하세요.

---

### 4. 갤럭시 웨어러블 앱으로 소프트웨어 업데이트 및 연결

- 새 이어버드와 기존 이어버드 양쪽의 **소프트웨어 버전이 동일하지 않으면 페어링 모드 진입이 안 됩니다**.
- 양쪽 이어버드를 케이스에 넣고 뚜껑을 닫았다가 다시 열어 모바일 기기의 Galaxy Wearable 앱에서 블루투스로 직접 연결하세요.
- 연결 후 이어버드 설정에서 소프트웨어 업데이트를 진행하면 자동으로 페어링 모드로 진입합니다.

---

### 5. 서비스 센터 방문 필요 시

- 만약 위 방법으로도 커플링이 안 되는 경우, 특히 모델별로는 서비스 센터에서 엔지니어의 도움을 받아야 할 수 있습니다.
- 서비스 접수 후 잃어버린 쪽의 이어버드를 구입하고, 엔지니어가 양쪽 이어버드 커플링 작업을 진행합니다.

---

## 주의사항
- 한쪽 이어버드만 새로 구매 시 **기존 이어버드와 동일한 배터리 타입인지 반드시 확인**하세요. (예: 2019년 9월 이전과 이후 생산 제품 간 배터리 타입 차이)
- 배터리 타입이 다르면 커플링이 불가능합니다.
- 이어버드와 케이스 접점에 이물질이나 부식이 있으면 정상 페어링이 어려울 수 있으니 깨끗하게 관리해 주세요.
- 소프트웨어 버전 차이로 페어링이 안 될 수 있으므로, 소프트웨어 업데이트를 꼭 진행해야 합니다.

---

필요시 삼성전자 서비스센터 방문을 권장드리며, 추가적인 문의가 있으면 언제든지 도움 드리겠습니다!

감사합니다.

Original Input: 삼성전자 S25 무게가 S24와 비교 했을때 얼마나 차이나
Google Search Query: 삼성전자 S25 무게 삼성전자 S24 무게 차이
LLM Query: 삼성전자 S25 모델의 무게가 삼성전자 S24 모델의 무게와 비교했을 때 얼마나 차이나는지 구체적으로 알려주세요. 두 모델의 무게 차이와 관련된 정보를 상세히 설명해 주세요.
Google Search Results: 5


삼성전자 갤럭시 S25 모델과 이전 모델인 갤럭시 S24 모델의 무게 차이에 대해 상세히 안내해 드리겠습니다.

---

## 갤럭시 S25 모델 무게
- **갤럭시 S25** : 162g
- **갤럭시 S25+ (플러스)** : 190g
- **갤럭시 S25 울트라** : 218g

---

## 갤럭시 S24 모델 무게 (참고)
- 갤럭시 S24 공식 무게는 약 **168g**으로 알려져 있습니다.

(※ S24 플러스 및 울트라 모델 무게는 각각 대략 S25 시리즈와 비슷하거나 약간 다를 수 있으나, 질문에서는 기본형 모델 위주로 비교합니다.)

---

## 무게 차이 및 설명
- 갤럭시 S25 기본 모델은 **162g**으로, 갤럭시 S24 기본 모델 약 **168g**보다 약 **6g 가벼워졌습니다**.
- 이는 S25 모델에서 둥근 모서리 디자인 채택과 내부 부품 최적화, 3나노 공정 AP 탑재 등으로 무게를 줄이면서도 성능을 강화한 결과로 볼 수 있습니다.

---

## 요약
| 모델명        | 무게    | S24 대비 무게 차이       |
|---------------|---------|--------------------------|
| 갤럭시 S24    | 약 168g | -                        |
| 갤럭시 S25    | 162g    | 약 6g 가벼워짐           |

---

더 궁금하신 점이나 다른 모델 비교 문의가 있으시면 언제든지 말씀해 주세요!